<a href="https://colab.research.google.com/github/shaligah/ETL-Pipeline-Data-Warehouse/blob/main/Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/shaligah/ETL-Pipeline-Data-Warehouse.git
%cd ETL-Pipeline-Data-Warehouse

Cloning into 'ETL-Pipeline-Data-Warehouse'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 50 (delta 13), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (50/50), 38.83 KiB | 883.00 KiB/s, done.
Resolving deltas: 100% (13/13), done.
/content/ETL-Pipeline-Data-Warehouse


In [2]:
!pip install sqlalchemy mlflow schedule --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 14.9 MB/s eta 0:00:00


In [3]:
import pandas as pd
import requests
from sqlalchemy import create_engine, Column, String, Integer, Float
from sqlalchemy.orm import declarative_base, sessionmaker
import sqlite3
import mlflow
import schedule
import time
from components import extract,transform_results,transform_drivers,transform_circuits,load_data

In [4]:
##Creating the DataBase for the first time

engine = create_engine('sqlite:///f1.db')

Base = declarative_base()
# creating the tables in the database
class Circuits(Base):
    __tablename__ = 'Circuits'

    CircuitId = Column(String, nullable=False,primary_key=True)
    CircuitName= Column(String, nullable=False)
    Country = Column(String, nullable=False)
    City = Column(String, nullable=False)
    Longitude = Column(Float)
    Latitude = Column(Float)
    Season = Column(Integer)

class Drivers(Base):
    __tablename__ = 'Drivers'

    DriverId = Column(String, nullable=False,primary_key=True)
    DriverName= Column(String, nullable=False)
    DOB = Column(String, nullable=False)
    Nationality = Column(String, nullable=False)
    Number = Column(Integer)
    Season = Column(Integer)

class Results(Base):
    __tablename__ = 'Results'

    id = Column(Integer, autoincrement=True, primary_key = True)
    RaceName = Column(String, nullable=False)
    Round= Column(Integer)
    Season = Column(Integer)
    CircuitName = Column(String, nullable=False)
    DriverName = Column(String, nullable=False)
    grid = Column(Integer)
    laps = Column(Integer)
    status = Column(String, nullable=False)
    position = Column(Integer)
    Time = Column(String, nullable=True)
    Constructor = Column(String, nullable=False)


Base.metadata.create_all(engine)

print('Warehouse successfully created  🔥')

Warehouse successfully created  🔥


In [5]:
#Creating the ETL Pipeline

def warehouse_pipeline():
    Session = sessionmaker(bind=engine)  # establisha connection to the database
    session = Session()
    #mlflow.set_tracking_uri("http://127.0.0.1:5000/")
    #mlflow.set_experiment("ETL")
    with mlflow.start_run(run_name = 'Warehouse_Pipeline') as run:  #using mlflow to log 5
        mlflow.set_tag('pipeline run', 'full run')

        print('Extracting the data...🚀')
        raw_data = extract
        mlflow.log_metric('rows extracted', len(raw_data))

        print('Transforming the data...🚀')
        results = transform_results(raw_data)
        mlflow.log_dict({'columns extracted' :list(results.columns)}, "results.json")

        drivers = transform_drivers(raw_data)
        mlflow.log_dict({'columns extracted' :list(drivers.columns)}, "drivers.json")

        circuits = transform_circuits(raw_data)
        mlflow.log_dict({'columns extracted' :list(circuits.columns)}, "circuits.json")

        print('Loading data into warehouse...🚀')
        load_data([
            (results,Results),
            (drivers, Drivers),
            (circuits, Circuits)
        ], session)

        print('Pipeline successfully created 🔥')
        mlflow.set_tag('status', 'completed')
    session.close() # close the connection

if __name__ == "__main__":
    warehouse_pipeline()


Extracting the data...🚀
Transforming the data...🚀
Loading data into warehouse...🚀
Pipeline successfully created 🔥


In [6]:
# run the ETL Pipeline evey tuesday at 1:15pm
"""
schedule.every().tuesday.at("13:15").do(warehouse_pipeline)


while True:
    schedule.run_pending()
    time.sleep(30)
"""

'\nschedule.every().tuesday.at("13:15").do(warehouse_pipeline)\n\n\nwhile True:\n    schedule.run_pending()\n    time.sleep(30)\n'

In [7]:
engine.dispose()    #useful as a last resort before deleting the database

In [10]:
#deleting the databse
"""
import os

db_file = "f1.db"

if os.path.exists(db_file):
    os.remove(db_file)
    print("Database deleted.")
else:
    print("Database does not exist.")
"""

Database deleted.
